# Caravan Club POI Generator 2020

In [53]:
# Install any missing libraries
!pip install simplekml
!pip install pandas
!pip install folium

In [54]:
# imports
import pandas as pd
import simplekml
import folium

In [49]:
df = pd.read_csv("data/cc-sites-2020.csv", header=0)
print(df.head(5))
print(df['Address'].head())

   Longitude   Latitude                                            Address
0   0.603700  51.264100  "Bearsted Caravan Club Site>01622 730018, Ashf...
1  -5.484430  55.583630  "Carradale Bay Caravan Site>01583 431665, Carr...
2  -3.473480  51.197940  "Minehead Caravan Club Site>01643 704345, Hopc...
3  -2.378320  52.527050  "Stanmore Hall Touring Park>01746 761761, Stou...
4  -0.664759  52.960932  "Wagtail Country Park>0781 448 1088, Cliff Lan...
0    "Bearsted Caravan Club Site>01622 730018, Ashf...
1    "Carradale Bay Caravan Site>01583 431665, Carr...
2    "Minehead Caravan Club Site>01643 704345, Hopc...
3    "Stanmore Hall Touring Park>01746 761761, Stou...
4    "Wagtail Country Park>0781 448 1088, Cliff Lan...
Name: Address, dtype: object


### Extract Site Name, Telephone Number, and Address in to their own columns

In [50]:
df['SiteName'] = df['Address'].str.extract('([a-zA-Z].*(?=>))')
df['PhoneNumber'] = df['Address'].str.extract('([0-9 ]+(?=,))')
# temp data frame with split value columns 
temp = df["Address"].str.split(",", n = 1, expand = True) 

# making separate site address column from new data frame 
df["SiteAddress"]= temp[1]
    
# Dropping old Address columns 
df.drop(columns =["Address"], inplace = True) 
 
# Remove extra quote the caravan club add to the site address field!
df['SiteAddress'] = df.apply(lambda x: x['SiteAddress'][:-1], axis=1)

# df display 
df 

,Longitude,Latitude,SiteName,PhoneNumber,SiteAddress
0,0.603700,51.264100,Bearsted Caravan Club Site,01622 730018,"Ashford Road, Hollingbourne, Maidstone, Kent,..."
1,-5.484430,55.583630,Carradale Bay Caravan Site,01583 431665,"Carradale, Campbeltown, Argyll & Bute, PA28 6..."
2,-3.473480,51.197940,Minehead Caravan Club Site,01643 704345,"Hopcott Road, Minehead, Somerset, TA24 6DJ, E..."
3,-2.378320,52.527050,Stanmore Hall Touring Park,01746 761761,"Stourbridge Road, Bridgnorth, Shropshire, WV1..."
4,-0.664759,52.960932,Wagtail Country Park,0781 448 1088,"Cliff Lane, Marston, Lincolnshire, NG32 2HU, ..."
...,...,...,...,...,...
191,-2.714524,52.341034,Ludlow Touring Park,01584 878788,"Overton Road, Ludlow, Shropshire, SY8 4AD, En..."
192,-2.157400,51.989120,Tewkesbury Abbey Caravan Club Site,01684 294035,"Gander Lane, Tewkesbury, Gloucestershire, GL2..."
193,-1.463928,55.076008,Old Hartley Caravan Club Site,0191 237 0256,"Whitley Bay, Tyne & Wear, NE26 4RL, England"
194,1.733200,52.628000,Great Yarmouth Racecourse Caravan Club Site,01493 855223,"Jellicoe Road, Great Yarmouth, Norfolk, NR30 ..."


In [56]:
# Convert the phone number to a string so we can add it to folium and kml
df['PhoneNumber'] = df['PhoneNumber'].astype(str)
df['PhoneNumber']


0       01622 730018
1       01583 431665
2       01643 704345
3       01746 761761
4      0781 448 1088
           ...      
191     01584 878788
192     01684 294035
193    0191 237 0256
194     01493 855223
195     01874 623325
Name: PhoneNumber, Length: 196, dtype: object

### Now create kml file for use by maps.me

### Once the .kml file has been created for maps.me will need to edit the file and change all instances of <styleUrl>#<number></styleUrl> with a valid maps.me styleUrl
    
<styleUrl>#placemark-blue</styleUrl>
<styleUrl>#placemark-red</styleUrl>
<styleUrl>#placemark-green</styleUrl>
<styleUrl>#placemark-yellow</styleUrl>
<styleUrl>#placemark-brown</styleUrl>
<styleUrl>#placemark-orange</styleUrl>
<styleUrl>#placemark-pink</styleUrl>    
<styleUrl>#placemark-purple</styleUrl>

In [75]:
kml = simplekml.Kml()
style = simplekml.Style()
style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/paddle/blu-blank.png'
style.iconstyle.scale = 1

map = folium.Map(location=[54.0, -1.0], zoom_start=6)
for index, row in df.iterrows():
    #name = row['SiteName'] + ', ' + row['SiteAddress'] + ', ' + row['PhoneNumber']
    name = row['SiteName']
    address = row['SiteAddress']+ ', ' + row['PhoneNumber']
    phonenumber = row['PhoneNumber']
    coords = [(row['Longitude'], row['Latitude'])]
    pnt = kml.newpoint(name=name)
    pnt.coords = coords
    pnt.style = style
    pnt.phonenumber = phonenumber
    pnt.description = address
    
    popupText = folium.Html('<p>'+row['SiteName']+'</p><p>'+row['SiteAddress']+'</p><p>'+row['PhoneNumber']+'</p>', script=True)
    #popupText = folium.Html('<p>'+row['SiteName']+'</p><p>'+row['SiteAddress']+'</p>', script=True)
    popup = folium.Popup(popupText, max_width=300)
    folium.Marker([row['Latitude'], row['Longitude']],
                 popup=popup,
                 icon=folium.Icon(color='red', icon='info-sign')).add_to(map)
    
kml.save(path="poi/cc-sites-2020.kml")
map

In [46]:
#!pip list